In [1]:
import numpy as np
from itertools import product
from concurrent.futures import ProcessPoolExecutor
from functools import partial
import time
import pandas as pd
import os

In [2]:
import numpy as np
import pandas as pd
from itertools import product
from multiprocessing import Pool, cpu_count
from tqdm import tqdm

# 固定参数
fixed_params = {
    'lambda_C': 0.5,   # 病毒清除率
    'd_T': 0.5,        # 未感染T细胞死亡率
    'eta': 0.5,        # 感染效率
#     'C_max': 5,        # 最大细胞浓度
    'S_T': 0.5,        # 目标细胞稳态浓度
    'a_T': 0.5         # 新增参数
}

# 可变参数范围（生成网格）
r_values = np.arange(1, 5, 0.5)        # 病毒复制率
d_M_values = np.arange(0.01, 1, 0.03)      # 感染细胞死亡率
S_M_values = np.arange(3, 5, 0.05)        # 感染细胞稳态浓度
q_values = np.arange(0.1, 5, 0.5)        # 病毒产生率
alpha_values = np.arange(0.1, 2, 0.5)    # 免疫反应强度
C_max_values = np.arange(1, 10, 0.5)    # 免疫反应强度


# 预计算所有参数组合的网格
param_grid = list(product(r_values, d_M_values, S_M_values, q_values, alpha_values,C_max_values))

def compute_coefficients(params):
    r_val, d_M_val, S_M_val, q_val, alpha_val,C_max = params
    lambda_C = fixed_params['lambda_C']
    d_T = fixed_params['d_T']
    eta = fixed_params['eta']
#     C_max = fixed_params['C_max']
    S_T = fixed_params['S_T']
    a_T = fixed_params['a_T']

    # 计算系数 a, b, c, d
    a = -d_M_val * S_T * lambda_C + r_val * S_M_val * lambda_C
    if a == 0:
        return None

    b = (d_M_val * S_T * lambda_C * C_max - d_M_val * S_T * lambda_C * q_val +
         r_val * S_M_val * q_val * lambda_C - r_val * S_M_val * lambda_C * C_max +
         d_M_val * d_T * lambda_C + alpha_val * r_val * lambda_C)

    c = (a_T * d_M_val * eta * C_max + d_M_val * S_T * lambda_C * C_max * q_val -
         r_val * S_M_val * q_val * lambda_C * C_max - d_M_val * d_T * lambda_C * C_max +
         d_M_val * d_T * lambda_C * q_val - alpha_val * r_val * lambda_C * C_max)

    d = a_T * d_M_val * eta * C_max * q_val - d_M_val * d_T * lambda_C * q_val * C_max

    # 条件3: g(0) = d/a < 0
    if (d / a) >= 0:
        return None

    # 条件1: Delta > 0
    Delta = (2 * (b / a))**2 - 4 * 3 * (c / a)
    if Delta <= 0:
        return None

    # 计算极值点 x1 < x2
    x1 = (-2 * (b / a) - np.sqrt(Delta)) / (2 * 3)
    x2 = (-2 * (b / a) + np.sqrt(Delta)) / (2 * 3)
    if x1 < 0:
        return None

    # 条件2: g(x1) > 0 且 g(x2) < 0
    g_x1 = x1**3 + (b / a) * x1**2 + (c / a) * x1 + (d / a)
    g_x2 = x2**3 + (b / a) * x2**2 + (c / a) * x2 + (d / a)
    if not (g_x1 > 0 and g_x2 < 0):
        return None
    
    # 计算三个正根
    roots = np.roots([1, b/a, c/a, d/a])
    real_roots = roots[np.isreal(roots)].real
    positive_roots = real_roots[real_roots > 0]
    
    # 条件4: 所有正根必须小于C_max
    if len(positive_roots) != 3 or (positive_roots >= C_max).any():
        return None

    # 构建结果字典
    result = {
        'r': r_val,
        'd_M': d_M_val,
        'S_M': S_M_val,
        'q': q_val,
        'alpha': alpha_val,
        'C_max': C_max,
        **fixed_params,
        'a': a,
        'b': b,
        'c': c,
        'd': d,
        'x1': x1,
        'x2': x2,
        'root1': positive_roots[0], 
        'root2': positive_roots[1],
        'root3': positive_roots[2]
    }
    
    # 立即打印找到的参数
    print(f"\n找到有效参数组合:")
    print(f"r={r_val:.2f}, d_M={d_M_val:.2f}, S_M={S_M_val:.2f}, q={q_val:.2f}, alpha={alpha_val:.2f}, C_max={C_max:.2f}")
    print(f"根: {positive_roots[0]:.2f}, {positive_roots[1]:.2f}, {positive_roots[2]:.2f}")
    
    return result

if __name__ == '__main__':
    # 并行计算（使用所有CPU核心）
    with Pool(cpu_count()) as pool:
        results = []
        # 使用imap_unordered更快获取结果
        with tqdm(total=len(param_grid), desc="搜索参数组合") as pbar:
            for result in pool.imap_unordered(compute_coefficients, param_grid):
                if result is not None:
                    results.append(result)
                pbar.update(1)
    
    # 保存结果
    if results:
        df = pd.DataFrame(results)
        df.to_csv('./data/valid_parameters.csv', index=False)
        print(f"\n搜索完成！共找到 {len(results)} 组有效参数，已保存到 valid_parameters.csv")
        
        # 打印汇总信息
        print("\n参数范围统计:")
        print(df[['r', 'd_M', 'S_M', 'q', 'alpha']].describe())
    else:
        print("\n未找到有效参数组合")

搜索参数组合: 100%|██████████████| 25872000/25872000 [08:47<00:00, 49033.52it/s]



未找到有效参数组合


In [4]:
import numpy as np
import pandas as pd
from itertools import product
from multiprocessing import Pool, cpu_count
from tqdm import tqdm
import os

# 固定参数
fixed_params = {
    'lambda_C': 0.5,   # 病毒清除率
    'd_T': 0.5,        # 未感染T细胞死亡率
    'eta': 0.5,        # 感染效率
    'S_T': 0.5,        # 目标细胞稳态浓度
    'a_T': 0.5         # 新增参数
}

# 可变参数范围（生成网格）
r_values = np.arange(1, 5, 0.5)        # 病毒复制率
d_M_values = np.arange(0.1, 2, 0.5)  # 感染细胞死亡率
S_M_values = np.arange(3, 5, 0.05)     # 感染细胞稳态浓度
q_values = np.arange(0.1, 5, 0.5)      # 病毒产生率
alpha_values = np.arange(0.1, 2, 0.5)  # 免疫反应强度
C_max_values = np.arange(1, 10, 0.5)   # 免疫反应强度

# 输出目录
output_dir = './data/parameter_batches'
os.makedirs(output_dir, exist_ok=True)

def generate_param_batches(batch_size=100000):
    """生成参数批次"""
    param_grid = product(r_values, d_M_values, S_M_values, q_values, alpha_values, C_max_values)
    batch = []
    for params in param_grid:
        batch.append(params)
        if len(batch) >= batch_size:
            yield batch
            batch = []
    if batch:  # 处理最后不足一个批次的部分
        yield batch

def process_batch(batch):
    """处理单个参数批次"""
    results = []
    for params in batch:
        result = compute_coefficients(params)
        if result is not None:
            results.append(result)
    return results

def compute_coefficients(params):
    r_val, d_M_val, S_M_val, q_val, alpha_val, C_max = params
    lambda_C = fixed_params['lambda_C']
    d_T = fixed_params['d_T']
    eta = fixed_params['eta']
    S_T = fixed_params['S_T']
    a_T = fixed_params['a_T']

    # 计算系数 a, b, c, d
    a = -d_M_val * S_T * lambda_C + r_val * S_M_val * lambda_C
    if a == 0:
        return None

    b = (d_M_val * S_T * lambda_C * C_max - d_M_val * S_T * lambda_C * q_val +
         r_val * S_M_val * q_val * lambda_C - r_val * S_M_val * lambda_C * C_max +
         d_M_val * d_T * lambda_C + alpha_val * r_val * lambda_C)

    c = (a_T * d_M_val * eta * C_max + d_M_val * S_T * lambda_C * C_max * q_val -
         r_val * S_M_val * q_val * lambda_C * C_max - d_M_val * d_T * lambda_C * C_max +
         d_M_val * d_T * lambda_C * q_val - alpha_val * r_val * lambda_C * C_max)

    d = a_T * d_M_val * eta * C_max * q_val - d_M_val * d_T * lambda_C * q_val * C_max

    # 条件3: g(0) = d/a < 0
    if (d / a) >= 0:
        return None

    # 条件1: Delta > 0
    Delta = (2 * (b / a))**2 - 4 * 3 * (c / a)
    if Delta <= 0:
        return None

    # 计算极值点 x1 < x2
    x1 = (-2 * (b / a) - np.sqrt(Delta)) / (2 * 3)
    x2 = (-2 * (b / a) + np.sqrt(Delta)) / (2 * 3)
    if x1 < 0:
        return None

    # 条件2: g(x1) > 0 且 g(x2) < 0
    g_x1 = x1**3 + (b / a) * x1**2 + (c / a) * x1 + (d / a)
    g_x2 = x2**3 + (b / a) * x2**2 + (c / a) * x2 + (d / a)
    if not (g_x1 > 0 and g_x2 < 0):
        return None
    
    # 计算三个正根
    roots = np.roots([1, b/a, c/a, d/a])
    real_roots = roots[np.isreal(roots)].real
    positive_roots = real_roots[real_roots > 0]
    
    # 条件4: 所有正根必须小于C_max
    if len(positive_roots) != 3 or (positive_roots >= C_max).any():
        return None

    # 构建结果字典
    result = {
        'r': r_val,
        'd_M': d_M_val,
        'S_M': S_M_val,
        'q': q_val,
        'alpha': alpha_val,
        'C_max': C_max,
        **fixed_params,
        'a': a,
        'b': b,
        'c': c,
        'd': d,
        'x1': x1,
        'x2': x2,
        'root1': positive_roots[0], 
        'root2': positive_roots[1],
        'root3': positive_roots[2]
    }
    
    return result

if __name__ == '__main__':
    batch_size = 10000000  # 根据内存调整批次大小
    total_batches = 0
    total_valid = 0
    
    # 创建批次生成器
    batch_generator = generate_param_batches(batch_size)
    
    # 处理每个批次
    for batch_idx, batch in enumerate(batch_generator):
        print(f"\n处理批次 {batch_idx + 1} (大小: {len(batch)})")
        
        # 并行处理当前批次
        with Pool(cpu_count()) as pool:
            # 将批次分成更小的块以便进度条更新
            chunks = [batch[i:i + 1000] for i in range(0, len(batch), 1000)]
            results = []
            
            with tqdm(total=len(chunks), desc=f"批次 {batch_idx + 1} 进度") as pbar:
                for chunk_result in pool.imap_unordered(process_batch, chunks):
                    results.extend(chunk_result)
                    pbar.update(1)
        
        # 保存当前批次的结果
        if results:
            df = pd.DataFrame(results)
            batch_file = os.path.join(output_dir, f'batch_{batch_idx + 1}.csv')
            df.to_csv(batch_file, index=False)
            total_valid += len(results)
            print(f"批次 {batch_idx + 1} 找到 {len(results)} 有效参数，已保存到 {batch_file}")
        
        total_batches = batch_idx + 1
        del batch, results  # 显式释放内存
        # 强制垃圾回收
        import gc
        gc.collect()
    
    # 合并所有批次结果
    if total_valid > 0:
        print("\n合并所有批次结果...")
        all_results = []
        for batch_idx in range(total_batches):
            batch_file = os.path.join(output_dir, f'batch_{batch_idx + 1}.csv')
            if os.path.exists(batch_file):
                df = pd.read_csv(batch_file)
                all_results.append(df)
        
        if all_results:
            final_df = pd.concat(all_results, ignore_index=True)
            final_file = os.path.join(output_dir, 'final_parameters.csv')
            final_df.to_csv(final_file, index=False)
            print(f"\n搜索完成！共找到 {total_valid} 组有效参数，已保存到 {final_file}")
            
            # 打印汇总信息
            print("\n参数范围统计:")
            print(final_df[['r', 'd_M', 'S_M', 'q', 'alpha', 'C_max']].describe())
    else:
        print("\n未找到有效参数组合")


处理批次 1 (大小: 921600)


批次 1 进度: 100%|██████████████████████████| 922/922 [00:00<00:00, 4500.52it/s]



未找到有效参数组合


In [1]:
import numpy as np
import pandas as pd
from itertools import product
from multiprocessing import Pool, cpu_count
from tqdm import tqdm
import os
import gc

# 定义所有参数的范围（不再有固定参数）
param_ranges = {
    'r': np.arange(0.1, 5, 0.2),          # 病毒复制率
    'd_M': np.arange(0.1, 5, 0.2),       # 感染细胞死亡率
    'S_M': np.arange(0.1, 5, 0.5),        # 感染细胞稳态浓度
    'q': np.arange(0.1, 5, 0.5),         # 病毒产生率
    'alpha': np.arange(0.01, 5, 0.5),      # 免疫反应强度
    'C_max': np.arange(1, 10, 1),        # 最大细胞浓度
    'lambda_C': np.arange(0.1, 2, 0.5),    # 病毒清除率
    'd_T': np.arange(0.1, 2, 0.5),         # 未感染T细胞死亡率
    'eta': np.arange(0.1, 2, 0.5),         # 感染效率
    'S_T': np.arange(0.1, 2, 0.5),         # 目标细胞稳态浓度
    'a_T': np.arange(0.1, 2, 0.5)          # 新增参数
}

# 输出目录
output_dir = './data/full_parameter_search'
os.makedirs(output_dir, exist_ok=True)

def generate_param_batches(param_ranges, batch_size=50000):
    """生成参数批次"""
    param_names = list(param_ranges.keys())
    value_arrays = [param_ranges[name] for name in param_names]
    
    # 计算总参数组合数
    total_combinations = np.prod([len(arr) for arr in value_arrays])
    print(f"总参数组合数: {total_combinations:,}")
    
    param_grid = product(*value_arrays)
    batch = []
    for params in param_grid:
        batch.append(params)
        if len(batch) >= batch_size:
            yield batch, param_names
            batch = []
    if batch:  # 处理最后不足一个批次的部分
        yield batch, param_names

def process_batch(args):
    """处理单个参数批次"""
    batch, param_names = args
    results = []
    for params in batch:
        result = compute_coefficients(dict(zip(param_names, params)))
        if result is not None:
            results.append(result)
    return results

def compute_coefficients(params):
    """计算系数并检查条件"""
    # 从参数字典中提取所有参数
    r_val = params['r']
    d_M_val = params['d_M']
    S_M_val = params['S_M']
    q_val = params['q']
    alpha_val = params['alpha']
    C_max = params['C_max']
    lambda_C = params['lambda_C']
    d_T = params['d_T']
    eta = params['eta']
    S_T = params['S_T']
    a_T = params['a_T']

    # 计算系数 a, b, c, d
    a = -d_M_val * S_T * lambda_C + r_val * S_M_val * lambda_C
    if a == 0:
        return None

    b = (d_M_val * S_T * lambda_C * C_max - d_M_val * S_T * lambda_C * q_val +
         r_val * S_M_val * q_val * lambda_C - r_val * S_M_val * lambda_C * C_max +
         d_M_val * d_T * lambda_C + alpha_val * r_val * lambda_C)

    c = (a_T * d_M_val * eta * C_max + d_M_val * S_T * lambda_C * C_max * q_val -
         r_val * S_M_val * q_val * lambda_C * C_max - d_M_val * d_T * lambda_C * C_max +
         d_M_val * d_T * lambda_C * q_val - alpha_val * r_val * lambda_C * C_max)

    d = a_T * d_M_val * eta * C_max * q_val - d_M_val * d_T * lambda_C * q_val * C_max

    # 条件3: g(0) = d/a < 0
    if (d / a) >= 0:
        return None

    # 条件1: Delta > 0
    Delta = (2 * (b / a))**2 - 4 * 3 * (c / a)
    if Delta <= 0:
        return None

    # 计算极值点 x1 < x2
    x1 = (-2 * (b / a) - np.sqrt(Delta)) / (2 * 3)
    x2 = (-2 * (b / a) + np.sqrt(Delta)) / (2 * 3)
    if x1 < 0:
        return None

    # 条件2: g(x1) > 0 且 g(x2) < 0
    g_x1 = x1**3 + (b / a) * x1**2 + (c / a) * x1 + (d / a)
    g_x2 = x2**3 + (b / a) * x2**2 + (c / a) * x2 + (d / a)
    if not (g_x1 > 0 and g_x2 < 0):
        return None
    
    # 计算三个正根
    roots = np.roots([1, b/a, c/a, d/a])
    real_roots = roots[np.isreal(roots)].real
    positive_roots = real_roots[real_roots > 0]
    
    # 条件4: 所有正根必须小于C_max
    if len(positive_roots) != 3 or (positive_roots >= C_max).any():
        return None

    # 构建结果字典（包含所有参数）
    result = params.copy()
    result.update({
        'a': a,
        'b': b,
        'c': c,
        'd': d,
        'x1': x1,
        'x2': x2,
        'root1': positive_roots[0], 
        'root2': positive_roots[1],
        'root3': positive_roots[2]
    })
    
    return result

if __name__ == '__main__':
    batch_size = 10000000  # 根据内存调整批次大小
    total_batches = 0
    total_valid = 0
    
    # 创建批次生成器
    batch_generator = generate_param_batches(param_ranges, batch_size)
    
    # 处理每个批次
    for batch_idx, (batch, param_names) in enumerate(batch_generator):
        print(f"\n处理批次 {batch_idx + 1} (大小: {len(batch)})")
        
        # 并行处理当前批次
        with Pool(cpu_count()) as pool:
            # 将批次分成更小的块以便进度条更新
            chunk_size = min(100000, len(batch) // cpu_count())
            chunks = [(batch[i:i + chunk_size], param_names) 
                     for i in range(0, len(batch), chunk_size)]
            
            results = []
            with tqdm(total=len(chunks), desc=f"批次 {batch_idx + 1} 进度") as pbar:
                for chunk_result in pool.imap_unordered(process_batch, chunks):
                    results.extend(chunk_result)
                    pbar.update(1)
        
        # 保存当前批次的结果
        if results:
            df = pd.DataFrame(results)
            batch_file = os.path.join(output_dir, f'batch_{batch_idx + 1:04d}.csv')
            df.to_csv(batch_file, index=False)
            total_valid += len(results)
            print(f"批次 {batch_idx + 1} 找到 {len(results)} 有效参数，已保存到 {batch_file}")
        
        total_batches = batch_idx + 1
        del batch, results  # 显式释放内存
        gc.collect()  # 强制垃圾回收
    
    # 合并所有批次结果
    if total_valid > 0:
        print("\n合并所有批次结果...")
        all_dfs = []
        for batch_idx in range(total_batches):
            batch_file = os.path.join(output_dir, f'batch_{batch_idx + 1:04d}.csv')
            if os.path.exists(batch_file):
                try:
                    df = pd.read_csv(batch_file)
                    all_dfs.append(df)
                    print(f"已加载批次 {batch_idx + 1} ({len(df)} 行)")
                except Exception as e:
                    print(f"加载批次 {batch_idx + 1} 时出错: {str(e)}")
        
        if all_dfs:
            final_df = pd.concat(all_dfs, ignore_index=True)
            final_file = os.path.join(output_dir, 'final_parameters.csv')
            final_df.to_csv(final_file, index=False)
            print(f"\n搜索完成！共找到 {total_valid} 组有效参数，已保存到 {final_file}")
            
            # 打印汇总信息
            print("\n参数范围统计:")
            print(final_df.describe())
            
            # 保存统计摘要
            stats_file = os.path.join(output_dir, 'parameter_stats.txt')
            with open(stats_file, 'w') as f:
                f.write(str(final_df.describe()))
    else:
        print("\n未找到有效参数组合")

总参数组合数: 5,760,000,000

处理批次 1 (大小: 10000000)


批次 1 进度: 100%|████████████████████████████| 100/100 [00:02<00:00, 39.80it/s]



处理批次 2 (大小: 10000000)


批次 2 进度: 100%|████████████████████████████| 100/100 [00:02<00:00, 38.04it/s]



处理批次 3 (大小: 10000000)


...


处理批次 575 (大小: 10000000)


批次 575 进度: 100%|██████████████████████████| 100/100 [00:02<00:00, 37.10it/s]



处理批次 576 (大小: 10000000)


批次 576 进度: 100%|██████████████████████████| 100/100 [00:02<00:00, 39.21it/s]



未找到有效参数组合
